In [78]:
# Clear all variables
from IPython import get_ipython
get_ipython().magic('reset -sf')



#include all the necessary libraries
from pile_utilities import generate_pile_coordinates



# Design Data Input Section

# Footing Data
concrete_fc_ksi = 5.5  # ksi
reinforcing_steel_fy_ksi = 60  # ksi
reinforcing_cover_in = 4.0  # in
column_x_dim_ft = 14.00  # ft
column_y_dim_ft = 9.00  # ft
column_ecc_x_ft = 0  # ft
column_ecc_y_ft = 0  # ft
footing_thickness_ft = 9  # ft
pile_embedment_in = 12  # in
pile_overhang_in = 19.5  # in
ground_elevation_ft = 73.4  # ft
footing_top_elevation_ft = 70.4  # ft
water_elevation_ft = 68.0  # ft
soil_weight_ksf = 0.115  # ksf
seal_thickness_ft = 0.00  # ft
pile_spacing_x_ft = 4.15  # ft
pile_spacing_y_ft = 3.75  # ft
pile_no_x = 8  # number of piles in x direction
pile_no_y = 6  # number of piles in y direction


# --- Pile Coordinates Generation and Table Display ---
pile_coordinates = generate_pile_coordinates(pile_no_x, pile_spacing_x_ft, pile_no_y, pile_spacing_y_ft)
# Use the already generated pile_coordinates variable (list of dicts)
# Display the pile coordinates in a formatted table
print("PileNo   x(ft)    y(ft)")
print("------------------------")
for pile in pile_coordinates:
    print(f"{pile['No.']:<8}{pile['x (ft)']:<9.2f}{pile['y (ft)']:<9.2f}")
# --- End of Pile Coordinates Section ---

# Pile Data Input Section
number_of_piles = pile_no_x*pile_no_y
pile_size_in = 15  # in
max_pile_driving_resistance_tons = 225  # tons
boring = "P41-1"
pile_tip_elevation_ft = 34  # ft
nominal_pile_bearing_capacity_tons = 270  # tons
soil_ultimate_side_friction_tons = 270  # tons
comp_reduction_factor = 0.75
uplift_reduction_factor = 0.6

PileNo   x(ft)    y(ft)
------------------------
1       0.00     0.00     
2       0.00     3.75     
3       0.00     7.50     
4       0.00     11.25    
5       0.00     15.00    
6       0.00     18.75    
7       4.15     0.00     
8       4.15     3.75     
9       4.15     7.50     
10      4.15     11.25    
11      4.15     15.00    
12      4.15     18.75    
13      8.30     0.00     
14      8.30     3.75     
15      8.30     7.50     
16      8.30     11.25    
17      8.30     15.00    
18      8.30     18.75    
19      12.45    0.00     
20      12.45    3.75     
21      12.45    7.50     
22      12.45    11.25    
23      12.45    15.00    
24      12.45    18.75    
25      16.60    0.00     
26      16.60    3.75     
27      16.60    7.50     
28      16.60    11.25    
29      16.60    15.00    
30      16.60    18.75    
31      20.75    0.00     
32      20.75    3.75     
33      20.75    7.50     
34      20.75    11.25    
35      20.75    15.00    
36    

C:\Users\lams6610\AppData\Local\Temp\ipykernel_36036\2034648248.py:3: DeprecationWarning:

`magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).



In [79]:
import math

# 1. Footing Longitudinal Dimension (ft)
Footing_x_ft = max(p['x (ft)'] for p in pile_coordinates) - min(p['x (ft)'] for p in pile_coordinates) + 2 * pile_overhang_in / 12

# 2. Footing Transverse Dimension (ft)
Footing_y_ft = max(p['y (ft)'] for p in pile_coordinates) - min(p['y (ft)'] for p in pile_coordinates) + 2 * pile_overhang_in / 12

# 3. Footing Effective Depth (de_ft)
D_ft = footing_thickness_ft  # in ft
dpile_in = pile_size_in      # in
bar_cover_in = reinforcing_cover_in  # in
bar_dia_in = 1.41            # in, assume #11 bar (change as needed)
de_ft = D_ft - dpile_in / 12 - bar_cover_in / 12 - bar_dia_in / 12  # in ft

# 4. Footing Shear Depth (dv_ft)

dv_ft = round(max(0.9 * de_ft, 0.72 * (D_ft - pile_embedment_in/12)), 3)

# 5. Column Effective Longitudinal Dimension (ft)
Col_x_ft = column_x_dim_ft

# 6. Column Effective Transverse Dimension (ft)
Col_y_ft = column_y_dim_ft 

# 7. Column Gross Area (ft^2)
Col_Area_ft2 = column_x_dim_ft * column_y_dim_ft

# 8. Footing + Seal Weight (kips)
Dseal_ft = seal_thickness_ft
W_footing_seal_kips = Footing_x_ft * Footing_y_ft * 0.15 * D_ft + Footing_x_ft * Footing_y_ft * Dseal_ft * 0.15

# 9. Soil Weight (kips)
Gamma_soil_ksf = soil_weight_ksf
G_Elev_ft = ground_elevation_ft
F_Elev_ft = footing_top_elevation_ft
W_soil_kips = (Footing_x_ft * Footing_y_ft - Col_Area_ft2) * Gamma_soil_ksf * max(G_Elev_ft - F_Elev_ft, 0)

# 10. Water Weight (kips)
W_Elev_ft = water_elevation_ft
water_unit_weight_kcf = 0.0624
water_depth_ft = max(min(W_Elev_ft, max(G_Elev_ft, F_Elev_ft)) - (F_Elev_ft - D_ft - Dseal_ft), 0)
W_water_kips = water_depth_ft * water_unit_weight_kcf * Footing_x_ft * Footing_y_ft

# Display results
print(f"Footing_x_ft: {Footing_x_ft:.2f} ft")
print(f"Footing_y_ft: {Footing_y_ft:.2f} ft")
print(f"Footing effective depth (de_ft): {de_ft:.2f} ft")
print(f"Footing shear depth (dv_ft): {dv_ft:.2f} ft")
print(f"Column effective longitudinal dimension (Col_x_ft): {Col_x_ft:.2f} ft")
print(f"Column effective transverse dimension (Col_y_ft): {Col_y_ft:.2f} ft")
print(f"Column gross area (Col_Area_ft2): {Col_Area_ft2:.2f} ft^2")
print(f"Footing + Seal Weight (W_footing_seal_kips): {W_footing_seal_kips:.2f} kips")
print(f"Soil Weight (W_soil_kips): {W_soil_kips:.2f} kips")
print(f"Water Weight (W_water_kips): {W_water_kips:.2f} kips")

Footing_x_ft: 32.30 ft
Footing_y_ft: 22.00 ft
Footing effective depth (de_ft): 7.30 ft
Footing shear depth (dv_ft): 6.57 ft
Column effective longitudinal dimension (Col_x_ft): 14.00 ft
Column effective transverse dimension (Col_y_ft): 9.00 ft
Column gross area (Col_Area_ft2): 126.00 ft^2
Footing + Seal Weight (W_footing_seal_kips): 959.31 kips
Soil Weight (W_soil_kips): 201.69 kips
Water Weight (W_water_kips): 292.65 kips


In [80]:
import pandas as pd

# Reactions at Top of Footing at Column Centroid (Footing Coordinate System)
# Placeholder values for demonstration; replace with actual load values as needed

# Create a DataFrame for the load cases
load_data = [
    [1, 1.00, 230, 233, 6764, 10600, 12710],
    [2, 1.00, -230, 233, 4876, 10590, 10160],
    [3, 1.00, 0, 143, 8113, 6958, 1782],
    [4, 1.00, -217, 143, 4611, 6943, 15900],
    [5, 1.00, 125, -322, 4876, 14160, 6856],
    [6, 1.00, 125, 322, 6764, 14120, 7270],
    [7, 1.00, 125, 322, 4864, 14120, 6853],
    [8, 1.00, 125, -322, 6764, 14170, 7270],
    [9, 1.00, -25, 180, 7977, 8880, 15700],
    [10, 1.00, -25, -180, 6089, 8524, 15290],
    [11, 1.00, -16, -173, 7557, 8451, 22680],
    [12, 1.00, -16, -173, 5657, 8459, 22150],
    [13, 1.00, 230, 233, 4113, 10590, 5152],
    [14, 1.00, -230, 233, 5704, 10590, 33970],
    [15, 1.00, 0, 143, 6841, 6951, 27300],
    [16, 1.00, -217, 143, 3848, 6939, 33340],
    [17, 1.00, -125, -322, 5692, 14160, 28530],
    [18, 1.00, -125, 322, 5704, 14130, 28530],
    [19, 1.00, -125, 322, 4101, 14130, 22160],
    [20, 1.00, -125, -322, 5704, 14160, 28530],
    [21, 1.00, -125, 322, 5704, 14120, 28530],
    [22, 1.00, -125, -322, 4101, 14160, 22160],
    [23, 1.00, 217, 143, 4366, 6943, 589],
    [24, 1.00, -217, 143, 5439, 6945, 39710],
    [25, 1.00, 230, -233, 2972, 11220, 2301],
    [26, 1.00, -230, -233, 2972, 11220, 23630],
    [27, 1.00, 0, -143, 3562, 7708, 14880],
    [28, 1.00, -217, -143, 1880, 7397, 25890],
    [29, 1.00, -125, -322, 2972, 13490, 18180],
    [30, 1.00, -125, 322, 2146, 14610, 14710],
    [31, 1.00, -125, 322, 2960, 14790, 18180],
    [32, 1.00, -125, -322, 2146, 13670, 14710],
    [33, 1.00, -125, 322, 2146, 14610, 14710],
    [34, 1.00, -125, -322, 2972, 13490, 18180],
    [35, 1.00, 217, -143, 2399, 7393, 8040],
    [36, 1.00, -217, -143, 2706, 7579, 29360],
]

columns = ["Load Case", "Factor", "Fx (k)", "Fy (k)", "Fz (k)", "Mx (k-ft)", "My (k-ft)"]
load_cases_df = pd.DataFrame(load_data, columns=columns)

# Display the DataFrame
display(load_cases_df)

,Load Case,Factor,Fx (k),Fy (k),Fz (k),Mx (k-ft),My (k-ft)
0,1,1.0,230,233,6764,10600,12710
1,2,1.0,-230,233,4876,10590,10160
2,3,1.0,0,143,8113,6958,1782
3,4,1.0,-217,143,4611,6943,15900
4,5,1.0,125,-322,4876,14160,6856
5,6,1.0,125,322,6764,14120,7270
6,7,1.0,125,322,4864,14120,6853
7,8,1.0,125,-322,6764,14170,7270
8,9,1.0,-25,180,7977,8880,15700
9,10,1.0,-25,-180,6089,8524,15290


In [81]:
# Define the `reactions` variable using the `load_cases_df` DataFrame
reactions = [
    {
        "load_case": row["Load Case"],
        "factor": row["Factor"],
        "fx": row["Fx (k)"],
        "fy": row["Fy (k)"],
        "fz": row["Fz (k)"],
        "mx": row["Mx (k-ft)"],
        "my": row["My (k-ft)"]
    }
    for _, row in load_cases_df.iterrows()
]


In [82]:
# Import necessary libraries
from collections import namedtuple

# Define a named tuple for design inputs
DesignInputs = namedtuple('DesignInputs', [
    's_x', 's_y', 'n_x', 'n_y', 'pile_overhang', 'footing_thickness',
    'col_x_dim', 'col_y_dim', 'soil_weight', 'ground_elev',
    'footing_top_elev', 'water_elev'
])

# Create an instance of DesignInputs
design_inputs = DesignInputs(
    s_x=pile_spacing_x_ft,
    s_y=pile_spacing_y_ft,
    n_x=pile_no_x,
    n_y=pile_no_y,
    pile_overhang=pile_overhang_in / 12,
    footing_thickness=footing_thickness_ft,
    col_x_dim=column_x_dim_ft,
    col_y_dim=column_y_dim_ft,
    soil_weight=soil_weight_ksf,
    ground_elev=ground_elevation_ft,
    footing_top_elev=footing_top_elevation_ft,
    water_elev=water_elevation_ft
)



import pandas as pd

# Precompute constants used in the axial load calculation
n_piles = len(pile_coordinates)

# Calculate Pcx and Pcy
Pcx = sum(p["x (ft)"] for p in pile_coordinates) / n_piles
Pcy = sum(p["y (ft)"] for p in pile_coordinates) / n_piles

# Display the calculated centroids
print(f"Pcx (Centroid in x-direction): {Pcx:.2f} ft")
print(f"Pcy (Centroid in y-direction): {Pcy:.2f} ft")

ix = sum((p["y (ft)"] - Pcx) ** 2 for p in pile_coordinates)
iy = sum((p["x (ft)"] - Pcy) ** 2 for p in pile_coordinates)

footing_length = design_inputs.s_x * (design_inputs.n_x - 1) + 2 * design_inputs.pile_overhang
footing_width = design_inputs.s_y * (design_inputs.n_y - 1) + 2 * design_inputs.pile_overhang
col_x = footing_length / 2
col_y = footing_width / 2
CCx = footing_length / 2 + column_ecc_x_ft
CCy = footing_width / 2 + column_ecc_y_ft
Fcx = footing_length / 2
Fcy = footing_width / 2


wt_foot = 0.15 * footing_length * footing_width * footing_thickness_ft
wt_soil = (footing_length * footing_width - design_inputs.col_x_dim * design_inputs.col_y_dim) * \
          design_inputs.soil_weight * max(design_inputs.ground_elev - design_inputs.footing_top_elev, 0)
wt_water = max(min(design_inputs.water_elev, max(design_inputs.ground_elev, design_inputs.footing_top_elev))
               - (design_inputs.footing_top_elev - footing_thickness_ft - 0), 0) * 0.0624 * footing_length * footing_width

forces_data = []


# Use the correct value from the 'Factor' column for each reaction
for reaction in reactions:
    dc = reaction["factor"]

    # Weight term
    if dc <= 1:
        weight_term = wt_foot - wt_water + wt_soil
    else:
        weight_term = wt_foot + wt_soil * 1.35 / 1.25

    # fz
    fz = reaction["fz"] + weight_term * dc

    # mx (corrected)
    mx = (reaction["mx"]
          + reaction["fy"] * (footing_thickness_ft - pile_embedment_in / 12)
          + reaction["fz"] * (CCy - Pcy)
          + weight_term * dc * (Fcy - Pcy))
    

    # my (corrected)
    my = (reaction["my"]
          + reaction["fx"] * (footing_thickness_ft - pile_embedment_in / 12)
          + reaction["fz"] * (Pcx - CCx)
          + weight_term * dc * (Pcx - Fcx))

    # Create a row for the DataFrame
    row = {
        "LC": reaction["load_case"],
        "DC": dc,
        "P": round(fz, 2),
        "Mx": round(mx, 2),
        "My": round(my, 2)
    }

    # Distribute forces to each pile
    for i, pile in enumerate(pile_coordinates, start=1):
        pile_x = pile["x (ft)"]
        pile_y = pile["y (ft)"]

        paxial = fz / n_piles
        pmy = my * (pile_x - Pcx) / ix if ix else 0
        pmx = mx * (pile_y - Pcy) / iy if iy else 0
        total_p = paxial + pmx + pmy

        row[str(i)] = round(total_p, 2)

    forces_data.append(row)



# Create the DataFrame for all data
columns_all = ["LC", "DC", "P", "Mx", "My"] + [str(i) for i in range(1, n_piles + 1)]
forces_df = pd.DataFrame(forces_data, columns=columns_all)

# Display the full table
print("Forces at Top of Pile Group Centroid and Axial Load on Individual Piles (All Data):")
display(forces_df)

# Calculate and display the max and min forces for each pile
print("\nUpdated Pile Forces:")
print("PileNo   Pmax (k)   Pmax Load Case   Pmin (k)   Pmin Load Case")
print("-----------------------------------------------------------")
for i, pile in enumerate(pile_coordinates, start=1):
    p_values = [row[str(i)] for row in forces_data]
    max_p = max(p_values)
    min_p = min(p_values)
    max_lc = forces_data[p_values.index(max_p)]["LC"]
    min_lc = forces_data[p_values.index(min_p)]["LC"]
    print(f"{i:<8}{max_p:<12.2f}{max_lc:<18}{min_p:<12.2f}{min_lc:<18}")


Pcx (Centroid in x-direction): 14.53 ft
Pcy (Centroid in y-direction): 9.38 ft
Forces at Top of Pile Group Centroid and Axial Load on Individual Piles (All Data):


,LC,DC,P,Mx,My,1,2,3,4,5,...,39,40,41,42,43,44,45,46,47,48
0,1.0,1.0,7632.34,24866.56,2147.44,107.85,124.47,141.08,157.69,174.30,...,157.57,174.19,190.80,207.41,127.10,143.71,160.32,176.94,193.55,210.16
1,2.0,1.0,5744.34,21788.56,-1014.56,87.83,102.38,116.94,131.50,146.05,...,109.15,123.71,138.26,152.82,78.74,93.29,107.85,122.41,136.96,151.52
2,3.0,1.0,8981.34,22696.68,-12812.68,206.61,221.77,236.94,252.10,267.26,...,138.52,153.69,168.85,184.01,91.80,106.96,122.12,137.29,152.45,167.61
3,4.0,1.0,5479.34,16990.93,5260.07,62.21,73.56,84.91,96.26,107.61,...,125.31,136.66,148.01,159.37,109.34,120.69,132.05,143.40,154.75,166.10
4,5.0,1.0,5744.34,20918.56,-1478.56,91.36,105.34,119.31,133.29,147.26,...,107.95,121.93,135.90,149.88,78.11,92.09,106.06,120.04,134.01,147.99
5,6.0,1.0,7632.34,29098.56,-4132.56,128.92,148.36,167.80,187.24,206.68,...,136.06,155.50,174.94,194.38,91.89,111.33,130.77,150.21,169.65,189.09
6,7.0,1.0,5732.34,26011.06,-1462.06,82.53,99.91,117.29,134.66,152.04,...,106.06,123.43,140.81,158.19,69.43,86.81,104.18,121.56,138.94,156.32
7,8.0,1.0,7632.34,23996.56,-4132.56,137.44,153.48,169.51,185.54,201.57,...,137.77,153.80,169.83,185.86,100.41,116.44,132.48,148.51,164.54,180.57
8,9.0,1.0,8845.34,24693.68,1126.32,137.99,154.49,170.98,187.48,203.98,...,179.63,196.13,212.63,229.13,148.08,164.58,181.08,197.57,214.07,230.57
9,10.0,1.0,6957.34,18389.68,3784.32,97.27,109.56,121.85,134.13,146.42,...,150.91,163.20,175.48,187.77,131.19,143.47,155.76,168.04,180.33,192.61



Updated Pile Forces:
PileNo   Pmax (k)   Pmax Load Case   Pmin (k)   Pmin Load Case
-----------------------------------------------------------
1       206.61      3.0               -48.86      28.0              
2       221.77      3.0               -41.70      28.0              
3       236.94      3.0               -34.54      28.0              
4       252.10      3.0               -27.37      28.0              
5       267.26      3.0               -20.21      28.0              
6       282.43      3.0               -13.05      28.0              
7       190.21      3.0               -23.65      28.0              
8       205.37      3.0               -16.49      28.0              
9       220.53      3.0               -9.33       28.0              
10      235.70      3.0               -2.17       28.0              
11      250.86      3.0               4.99        28.0              
12      266.02      3.0               12.15       28.0              
13      173.81      3.0    

In [83]:
# Add Pmax (k) and Pmin (k) to pile forces for visualization
pile_forces = [
    {
        "No.": pile["No."],
        "x (ft)": pile["x (ft)"],
        "y (ft)": pile["y (ft)"],
        "Pmax (k)": max(row[str(i)] for row in forces_data),
        "Pmin (k)": min(row[str(i)] for row in forces_data)
    }
    for i, pile in enumerate(pile_coordinates, start=1)
 ]

In [84]:
import plotly.graph_objects as go

# Ensure pile_center_x and pile_center_y are defined
pile_center_x = sum(p["x (ft)"] for p in pile_coordinates) / len(pile_coordinates)
pile_center_y = sum(p["y (ft)"] for p in pile_coordinates) / len(pile_coordinates)

def plot_interactive_forces_top_view(pile_forces, column_x_dim, column_y_dim, footing_x_dim, footing_y_dim, pile_center_x, pile_center_y):
    x = [p["x (ft)"] for p in pile_forces]
    y = [p["y (ft)"] for p in pile_forces]
    z = [p["Pmax (k)"] for p in pile_forces]

    # Create the plot
    fig = go.Figure()

    # Add pile forces as scatter points
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers+text',
        text=[f'P{p["No."]}: {p["Pmax (k)"]:.0f}k' for p in pile_forces],
        marker=dict(
            size=16,
            color=z,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Force (kips)')
        ),
        textposition='top center',
        name='Pile Forces'
    ))

    # Add the column trace as a rectangle
    fig.add_trace(go.Scatter(
        x=[
            pile_center_x - column_x_dim / 2,
            pile_center_x + column_x_dim / 2,
            pile_center_x + column_x_dim / 2,
            pile_center_x - column_x_dim / 2,
            pile_center_x - column_x_dim / 2
        ],
        y=[
            pile_center_y - column_y_dim / 2,
            pile_center_y - column_y_dim / 2,
            pile_center_y + column_y_dim / 2,
            pile_center_y + column_y_dim / 2,
            pile_center_y - column_y_dim / 2
        ],
        mode='lines',
        line=dict(color='red', width=2),
        name='Column'
    ))

    # Add the footing trace as a rectangle
    fig.add_trace(go.Scatter(
        x=[
            pile_center_x - footing_x_dim / 2,
            pile_center_x + footing_x_dim / 2,
            pile_center_x + footing_x_dim / 2,
            pile_center_x - footing_x_dim / 2,
            pile_center_x - footing_x_dim / 2
        ],
        y=[
            pile_center_y - footing_y_dim / 2,
            pile_center_y - footing_y_dim / 2,
            pile_center_y + footing_y_dim / 2,
            pile_center_y + footing_y_dim / 2,
            pile_center_y - footing_y_dim / 2
        ],
        mode='lines',
        line=dict(color='blue', width=2, dash='dash'),
        name='Footing'
    ))

    # Update layout
    fig.update_layout(
        title='Pile Force Distribution (Top View)',
        xaxis_title='X Distance (ft)',
        yaxis_title='Y Distance (ft)',
        width=800,
        height=800,
        yaxis=dict(scaleanchor="x", scaleratio=1),
        legend=dict(
            title="Legend",
            orientation="h",
            yanchor="bottom",
            y=1.01,
            xanchor="center",
            x=0.5,
            bordercolor="Black",
            borderwidth=1
        )
    )

    # Save the plot as a high-resolution image (600 DPI)
    # fig.write_image("pile_force_distribution.png", width=800, height=800, scale=6)  # Scale factor for 600 DPI

    # Show the plot
    fig.show()

# Call the updated function
plot_interactive_forces_top_view(
    pile_forces,
    column_x_dim_ft,
    column_y_dim_ft,
    Footing_x_ft,  # Footing dimensions
    Footing_y_ft,
    pile_center_x,
    pile_center_y
)

In [85]:
# Calculate total of maximum pile loads (Pmax) for piles before the face of column in x-axis (i.e., x < col_face_x)
# This is Vuy1

# Define col_face_x as the x-coordinate of the column's left face
col_face_x = pile_center_x - column_x_dim_ft / 2

# Define col_face_x_right as the x-coordinate of the column's right face
col_face_x_right = pile_center_x + column_x_dim_ft / 2

# Sum Pmax for all piles with x < col_face_x
Vuy1 = sum(pile['Pmax (k)'] for pile in pile_forces if pile['x (ft)'] < col_face_x)

print(f"Total maximum pile load before the face of column in x-axis (Vuy1): {Vuy1:.2f} kips")

# Sum Pmax for all piles with x > col_face_x_right
Vuy2 = sum(pile['Pmax (k)'] for pile in pile_forces if pile['x (ft)'] > col_face_x_right)

print(f"Total maximum pile load after the face of column in x-axis (Vuy2): {Vuy2:.2f} kips")

# Define col_face_y as the y-coordinate of the column's bottom face
col_face_y = pile_center_y - column_y_dim_ft / 2

# Define col_face_y_top as the y-coordinate of the column's top face
col_face_y_top = pile_center_y + column_y_dim_ft / 2

# Sum Pmax for all piles with y < col_face_y (below the column)
Vux1 = sum(pile['Pmax (k)'] for pile in pile_forces if pile['y (ft)'] < col_face_y)

# Sum Pmax for all piles with y > col_face_y_top (above the column)
Vux2 = sum(pile['Pmax (k)'] for pile in pile_forces if pile['y (ft)'] > col_face_y_top)

print(f"Total maximum pile load below the face of column in y-axis (Vux1): {Vux1:.2f} kips")
print(f"Total maximum pile load above the face of column in y-axis (Vux2): {Vux2:.2f} kips")



Total maximum pile load before the face of column in x-axis (Vuy1): 2835.80 kips
Total maximum pile load after the face of column in x-axis (Vuy2): 2854.53 kips
Total maximum pile load below the face of column in y-axis (Vux1): 3006.74 kips
Total maximum pile load above the face of column in y-axis (Vux2): 3928.45 kips


In [86]:
# Corrected Shear Capacity Calculation along X and Y directions
phi_v = 0.90  # Strength reduction factor for shear
beta = 2.0  # per LRFD 5.8.3.4.1

# Shear capacity along X direction
vc1_x = phi_v * 0.0316 * beta * math.sqrt(concrete_fc_ksi) * Footing_x_ft * dv_ft * 144  # Include *144 for unit conversion
vc2_x = 0.25 * concrete_fc_ksi * Footing_x_ft * dv_ft * 144
vn_x = min(vc1_x, vc2_x)  # Convert to kips

# Shear capacity along Y direction
vc1_y = phi_v * 0.0316 * beta * math.sqrt(concrete_fc_ksi) * Footing_y_ft * dv_ft * 144  # Include *144 for unit conversion
vc2_y = 0.25 * concrete_fc_ksi * footing_thickness_ft * dv_ft * 144
vn_y = min(vc1_y, vc2_y)   # Convert to kips

# Print results
print(f"Shear Capacity along X direction (Vn_x): {vn_x:.2f} kips")
print(f"Shear Capacity along Y direction (Vn_y): {vn_y:.2f} kips")

Shear Capacity along X direction (Vn_x): 4075.73 kips
Shear Capacity along Y direction (Vn_y): 2776.04 kips


In [87]:
from IPython.display import display, HTML

# Check one-way shear safety for transverse (y) and longitudinal (x) sections

# For transverse (y) direction
if Vuy1 < vn_y and Vuy2 < vn_y:
    display(HTML('<span style="color:green; font-weight:bold;">SAFE in one-way shear (transverse/y direction): Both Vuy1 and Vuy2 are less than Vn_y.</span>'))
else:
    if Vuy1 >= vn_y:
        print(f"NOT SAFE: Vuy1 ({Vuy1:.2f} kips) exceeds Vn_y ({vn_y:.2f} kips) in transverse/y direction.")
    if Vuy2 >= vn_y:
        print(f"NOT SAFE: Vuy2 ({Vuy2:.2f} kips) exceeds Vn_y ({vn_y:.2f} kips) in transverse/y direction.")

# For longitudinal (x) direction
if Vux1 < vn_x and Vux2 < vn_x:
    display(HTML('<span style="color:green; font-weight:bold;">SAFE in one-way shear (longitudinal/x direction): Both Vux1 and Vux2 are less than Vn_x.</span>'))
else:
    if Vux1 >= vn_x:
        print(f"NOT SAFE: Vux1 ({Vux1:.2f} kips) exceeds Vn_x ({vn_x:.2f} kips) in longitudinal/x direction.")
    if Vux2 >= vn_x:
        print(f"NOT SAFE: Vux2 ({Vux2:.2f} kips) exceeds Vn_x ({vn_x:.2f} kips) in longitudinal/x direction.")

NOT SAFE: Vuy1 (2835.80 kips) exceeds Vn_y (2776.04 kips) in transverse/y direction.
NOT SAFE: Vuy2 (2854.53 kips) exceeds Vn_y (2776.04 kips) in transverse/y direction.


In [88]:
  # Function to find corner piles and get their maximum load
def get_corner_pile_max_load(pile_forces):
        # Find min/max x and y coordinates
        xs = [p["x (ft)"] for p in pile_forces]
        ys = [p["y (ft)"] for p in pile_forces]
        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)
    
        # Identify corner piles (those at the 4 corners)
        corner_piles = [
            p for p in pile_forces
            if (p["x (ft)"] in [min_x, max_x]) and (p["y (ft)"] in [min_y, max_y])
        ]
    
        # Get their Pmax (k) values
        corner_pmax = [p["Pmax (k)"] for p in corner_piles]
        Vu = max(corner_pmax) if corner_pmax else 0
        return Vu, corner_piles
    
# Calculate Vu based on corner piles
Vu, corner_piles = get_corner_pile_max_load(pile_forces)

# Display corner piles and their maximum loads
print("Corner Piles and Their Maximum Loads:")
for pile in corner_piles:
        print(f"Pile No: {pile['No.']}, Pmax: {pile['Pmax (k)']} kips")
    
# Convert units
dv = dv_ft* 12  # in
dv2 = 0.5 * dv / 12  # ft
    
# Perimeter of critical section (in feet)
pile_size_ft = pile_size_in / 12  # Convert pile size to feet
pile_edge_ft = pile_overhang_in / 12  # Assuming pile edge is the overhang
b0_opt1 = 4 * (pile_size_ft + 2 * dv2)
b0_opt2 = pile_size_ft + 2 * (dv2 + pile_edge_ft)
b0 = min(b0_opt1, b0_opt2)  # ft
b0_in = b0 * 12  # Convert to inches
    
# Nominal shear resistance
beta_c = 2.0  # Assuming beta_c is 2.0 (adjust if needed)
fc = concrete_fc_ksi  # Concrete compressive strength in ksi
phi = 0.9  # Strength reduction factor for shear
    
Vn1 = (0.063 + 0.126 / beta_c) * math.sqrt(fc) * b0_in * dv  # kips
Vn2 = 0.126 * math.sqrt(fc) * b0_in * dv  # kips
Vn = min(Vn1, Vn2)
    
# Design shear capacity
phiVn = phi * Vn
    
# Status for shear check
status = "OK" if phiVn > Vu else "NG"
dc_ratio = Vu / phiVn if phiVn != 0 else float('inf')

# Display results
print(f"Shear Check: phiVn = {phiVn:.2f} kips, Vu = {Vu:.2f} kips, Status = {status}")
print(f"D/C Ratio (Vu / phiVn): {dc_ratio:.3f}")

Corner Piles and Their Maximum Loads:
Pile No: 1, Pmax: 206.61 kips
Pile No: 6, Pmax: 282.43 kips
Pile No: 43, Pmax: 224.99 kips
Pile No: 48, Pmax: 286.25 kips
Shear Check: phiVn = 2784.61 kips, Vu = 286.25 kips, Status = OK
D/C Ratio (Vu / phiVn): 0.103


In [89]:
def flexural_moment_check():
    # Flexural moment check along x-direction
    footing_dim_x = Footing_x_ft
    col_dim_x = column_x_dim_ft
    edge_x = (footing_dim_x - col_dim_x) / 2
    crit_x = edge_x - pile_overhang_in / 12  # Critical section distance in feet

    # Find critical pile group and Pu for x-direction
    col_x_min = pile_center_x - column_x_dim_ft / 2
    col_x_max = pile_center_x + column_x_dim_ft / 2
    piles_right = [
        i for i, coord in enumerate(pile_coordinates)
        if coord["x (ft)"] > col_x_max
    ]
    piles_left = [
        i for i, coord in enumerate(pile_coordinates)
        if coord["x (ft)"] < col_x_min
    ]
    Pu_right = sum(pile_forces[i]["Pmax (k)"] for i in piles_right)
    Pu_left = sum(pile_forces[i]["Pmax (k)"] for i in piles_left)
    Pu_x = max(Pu_right, Pu_left)
    Mpile_x = Pu_x * crit_x  # Moment from pile group in x-direction

    # Flexural moment check along y-direction
    footing_dim_y = Footing_y_ft
    col_dim_y = column_y_dim_ft
    edge_y = (footing_dim_y - col_dim_y) / 2
    crit_y = edge_y - pile_overhang_in / 12  # Critical section distance in feet

    # Find critical pile group and Pu for y-direction
    col_y_min = pile_center_y - column_y_dim_ft / 2
    col_y_max = pile_center_y + column_y_dim_ft / 2
    piles_above = [
        i for i, coord in enumerate(pile_coordinates)
        if coord["y (ft)"] > col_y_max
    ]
    piles_below = [
        i for i, coord in enumerate(pile_coordinates)
        if coord["y (ft)"] < col_y_min
    ]
    Pu_above = sum(pile_forces[i]["Pmax (k)"] for i in piles_above)
    Pu_below = sum(pile_forces[i]["Pmax (k)"] for i in piles_below)
    Pu_y = max(Pu_above, Pu_below)
    Mpile_y = Pu_y * crit_y  # Moment from pile group in y-direction

    # Moment from footing self-weight
    Mfooting_x = footing_dim_x * footing_thickness_ft * 0.15 * (edge_x**2 / 2)
    Mfooting_y = footing_dim_y * footing_thickness_ft * 0.15 * (edge_y**2 / 2)

    # Moment from surcharge (if any)
    Msurcharge_x = footing_dim_x * seal_thickness_ft * soil_weight_ksf * (edge_x**2 / 2)
    Msurcharge_y = footing_dim_y * seal_thickness_ft * soil_weight_ksf * (edge_y**2 / 2)

    # Total factored moments
    Mu_total_x = Mpile_x + Mfooting_x + Msurcharge_x
    Mu_total_y = Mpile_y + Mfooting_y + Msurcharge_y

    # Strength I and Service I design moments
    Mstrength_x = Mpile_x - 0.9 * Mfooting_x - 0.9 * Msurcharge_x
    Mstrength_y = Mpile_y - 0.9 * Mfooting_y - 0.9 * Msurcharge_y
    Mservice_x = Mpile_x - 1.0 * Mfooting_x - 1.0 * Msurcharge_x
    Mservice_y = Mpile_y - 1.0 * Mfooting_y - 1.0 * Msurcharge_y

    # Output results
    result_x = {
        "direction": "x",
        "x_edge (ft)": round(edge_x, 3),
        "x_crit (ft)": round(crit_x, 3),
        "Pu_x (kips)": round(Pu_x, 2),
        "Mpile_x (kip-ft)": round(Mpile_x, 2),
        "Mfooting_x (kip-ft)": round(Mfooting_x, 2),
        "Msurcharge_x (kip-ft)": round(Msurcharge_x, 2),
        "Mu_total_x (kip-ft)": round(Mu_total_x, 2),
        "Mstrength_I_x (kip-ft)": round(Mstrength_x, 2),
        "Mservice_I_x (kip-ft)": round(Mservice_x, 2),
    }

    result_y = {
        "direction": "y",
        "y_edge (ft)": round(edge_y, 3),
        "y_crit (ft)": round(crit_y, 3),
        "Pu_y (kips)": round(Pu_y, 2),
        "Mpile_y (kip-ft)": round(Mpile_y, 2),
        "Mfooting_y (kip-ft)": round(Mfooting_y, 2),
        "Msurcharge_y (kip-ft)": round(Msurcharge_y, 2),
        "Mu_total_y (kip-ft)": round(Mu_total_y, 2),
        "Mstrength_I_y (kip-ft)": round(Mstrength_y, 2),
        "Mservice_I_y (kip-ft)": round(Mservice_y, 2),
    }

    return result_x, result_y


# Call the function and display results
result_x, result_y = flexural_moment_check()
print("Flexural Moment Check - X Direction:")
for key, value in result_x.items():
    print(f"{key}: {value}")

print("\nFlexural Moment Check - Y Direction:")
for key, value in result_y.items():
    print(f"{key}: {value}")

Flexural Moment Check - X Direction:
direction: x
x_edge (ft): 9.15
x_crit (ft): 7.525
Pu_x (kips): 2854.53
Mpile_x (kip-ft): 21480.34
Mfooting_x (kip-ft): 1825.36
Msurcharge_x (kip-ft): 0.0
Mu_total_x (kip-ft): 23305.7
Mstrength_I_x (kip-ft): 19837.51
Mservice_I_x (kip-ft): 19654.98

Flexural Moment Check - Y Direction:
direction: y
y_edge (ft): 6.5
y_crit (ft): 4.875
Pu_y (kips): 3928.45
Mpile_y (kip-ft): 19151.19
Mfooting_y (kip-ft): 627.41
Msurcharge_y (kip-ft): 0.0
Mu_total_y (kip-ft): 19778.61
Mstrength_I_y (kip-ft): 18586.52
Mservice_I_y (kip-ft): 18523.78


In [90]:
# Multimate_x is the maximum of Mstrength_I_x and Mservice_I_x (from result_x)
Multimate_x = max(result_x["Mstrength_I_x (kip-ft)"], result_x["Mservice_I_x (kip-ft)"])

# Multimate_y is the maximum of Mstrength_I_y and Mservice_I_y (from result_y)
Multimate_y = max(result_y["Mstrength_I_y (kip-ft)"], result_y["Mservice_I_y (kip-ft)"])

print(f"Multimate_x (kip-ft): {Multimate_x}")
print(f"Multimate_y (kip-ft): {Multimate_y}")

Multimate_x_per_ft = Multimate_x / Footing_x_ft  # Convert to per foot

from math import sqrt  # Import sqrt from the math module

As_x = 0.85 * (concrete_fc_ksi / reinforcing_steel_fy_ksi) * Footing_x_ft * D_ft * (1 - sqrt(1 - (4 * Multimate_x_per_ft) / (1.7 * 0.9  * concrete_fc_ksi*144 * de_ft**2)))

print(f"Required reinforcement in x direction (As_x): {As_x:.2f} in^2")


# Calculate required reinforcement in y direction (As_y)
Multimate_y_per_ft = Multimate_y / Footing_y_ft  # Convert to per foot

As_y = 0.85 * (concrete_fc_ksi / reinforcing_steel_fy_ksi) * Footing_y_ft * D_ft * (1 - sqrt(1 - (4 * Multimate_y_per_ft) / (1.7 * 0.9 * concrete_fc_ksi * 144 * de_ft**2)))

print(f"Required reinforcement in y direction (As_y): {As_y:.2f} in^2")

Multimate_x (kip-ft): 19837.51
Multimate_y (kip-ft): 18586.52
Required reinforcement in x direction (As_x): 0.44 in^2
Required reinforcement in y direction (As_y): 0.41 in^2
